<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Checking-your-TensorFlow-version" data-toc-modified-id="Checking-your-TensorFlow-version-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Checking your TensorFlow version</a></span></li><li><span><a href="#Build-a-small-computation-graph-and-run-it" data-toc-modified-id="Build-a-small-computation-graph-and-run-it-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Build a small computation graph and run it</a></span></li><li><span><a href="#Get-a-value-from-a-tensor-op" data-toc-modified-id="Get-a-value-from-a-tensor-op-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Get a value from a tensor op</a></span></li><li><span><a href="#Reusing-the-computation-graph?" data-toc-modified-id="Reusing-the-computation-graph?-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Reusing the computation graph?</a></span></li><li><span><a href="#Building-a-reusable-computation-graph" data-toc-modified-id="Building-a-reusable-computation-graph-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Building a reusable computation graph</a></span></li><li><span><a href="#Function-decorator-tf.function" data-toc-modified-id="Function-decorator-tf.function-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Function decorator <code>tf.function</code></a></span></li></ul></div>

# Introduction

In October 2019 TensorFlow 2.0 was published. Things changed a little bit. So let us see here in this notebook how to define a computation graph in TensorFlow 2.0.

# Checking your TensorFlow version

In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.0.0


In [3]:
print("Eager execution: {}".format(tf.executing_eagerly()))

Eager execution: True


# Build a small computation graph and run it

In [4]:
a = tf.constant(3)
b = tf.constant(4)

In [5]:
print(a)

tf.Tensor(3, shape=(), dtype=int32)


In [6]:
a

<tf.Tensor: id=0, shape=(), dtype=int32, numpy=3>

In [7]:
print(type(a))

<class 'tensorflow.python.framework.ops.EagerTensor'>


In [8]:
c = a+b

In [9]:
print(type(c))

<class 'tensorflow.python.framework.ops.EagerTensor'>


In [10]:
c

<tf.Tensor: id=2, shape=(), dtype=int32, numpy=7>

In [11]:
print(c)

tf.Tensor(7, shape=(), dtype=int32)


So we can see, that the tensor op with id 2 already has computed the result: 7.

For this, the tensor ops are called "eager tensors".

Before, in TensorFlow 1.x, we had first to specify a graph and then run a session. That is no longer needed!

# Get a value from a tensor op

In [12]:
c

<tf.Tensor: id=2, shape=(), dtype=int32, numpy=7>

In [13]:
print(c)

tf.Tensor(7, shape=(), dtype=int32)


In [14]:
c.numpy()

7

In [15]:
type(c.numpy())

numpy.int32

In [16]:
c.numpy() * 2

14

# Reusing the computation graph?

So we have a simple graph with 3 nodes: `a`,`b` are TensorFlow constants and `c` is a TensorFlow operation for addition.

Can we reuse the graph with other input for `a` and `b`?

In [39]:
a = tf.constant(9)
b = tf.constant(10)

In [40]:
a

<tf.Tensor: id=59, shape=(), dtype=int32, numpy=9>

We can see, that the ID of the tensor changed. So we actually created two new nodes and did not change the values in the nodes.

In [41]:
c

<tf.Tensor: id=49, shape=(), dtype=int32, numpy=7>

The last command further showed us, that `c` has not recomputed its output. The value of `c` is still the old value "7" and not "19" (9+10).

# Building a reusable computation graph

In [81]:
a = tf.Variable(1)
b = tf.Variable(2)
c = a+b

In [82]:
c

<tf.Tensor: id=150, shape=(), dtype=int32, numpy=3>

In [83]:
a.assign(3)
b.assign(4)

<tf.Variable 'UnreadVariable' shape=() dtype=int32, numpy=4>

In [84]:
a

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=3>

In [85]:
b

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=4>

In [86]:
a.numpy()

3

In [87]:
b.numpy()

4

`a` and `b` have their new values! Great! But what is with `c`?

In [88]:
c

<tf.Tensor: id=150, shape=(), dtype=int32, numpy=3>

The tensor `c` has still its old value!

In [89]:
def compute():
    c = a + b
    return c

In [97]:
a.assign(5)
b.assign(6)
compute()

<tf.Tensor: id=191, shape=(), dtype=int32, numpy=11>

In [98]:
a.assign(7)
b.assign(8)
compute()

<tf.Tensor: id=194, shape=(), dtype=int32, numpy=15>

Now it works! Actually each time we call `compute()` a new TensorFlow operation node `c` is generated, which can see from the `id` attribute of the `c` TensorFlow op.

Now let us make a small change!

We decorate the function `compute()` with `@tf.function`:

In [99]:
@tf.function
def compute():
    c = a + b
    return c

In [100]:
a.assign(1)
b.assign(2)
compute()

<tf.Tensor: id=206, shape=(), dtype=int32, numpy=3>

In [101]:
a.assign(3)
b.assign(4)
compute()

<tf.Tensor: id=209, shape=(), dtype=int32, numpy=7>

So the question arises: what is the decorator `@tf.function` good for?

# Function decorator `tf.function`

From https://pgaleone.eu/tensorflow/tf.function/2019/03/21/dissecting-tf-function-part-1/

we can read:
    
    The automatic conversion from Python code to its graph representation is called AutoGraph.

    In Tensorflow 2.0, AutoGraph is automatically applied to a function when it is decorated with @tf.function; this decorator creates callable graphs from Python functions.
    
    tf.function: layman explanation

    On the first call of a tf.function decorated function:

    The function is executed and traced. Eager execution is disabled in this context, therefore every tf. method just define a tf.Operation node that produces a tf.Tensor output, Tensorflow 1.x like.
    
    AutoGraph is used to detect Python constructs that can be converted to their graph equivalent (while → tf.while, for → tf.while, if → tf.cond, assert → tf.assert, …).
    
    From the function trace + autograph, the graph representation is built. In order to preserve the execution order in the defined graph, tf.control_dependencies is automatically added after every statement, in order to condition the line i+1 on the execution of line i.
    
    The tf.Graph object has now been built.
    
    Based on the function name and the input parameters a unique ID is created and associated with the graph. The graph is cached into a map: map[id] = graph. Any function call will just re-use the defined graph if the key matches.

So in conclusion: the `@tf.function` decorator allows TensorFlow to generate a computation graph automatically (AutoGraph) and reuse it for iterative computations.

Iterative computations means: we want to run the graph again and again, but with other inputs, e.g. a CNN with different images as inputs.

In such a case, we do not want to build the graph from scratch each time when we have another image as input, right?